# Temperature trend evaluation

Since I have an interest in weather and climatology related phenomena, I've decided to do a small analysis of temperature trends. I'm using data from KNMI at it's base station 'De Bilt' as reference data. Data contains daily temperature (and other) data starting in 1901. It is now May 2020, and at the end of the year the KNMI will come up with new climate normals, as those are reset every 10 years. The last batch is from 1981 - 2010. This is a very simple exercise but purely doing it out of my own interest, I will answer the questions:

- What will be the new normals for the 1991 - 2020 period (discarding the period of May 2020 - December 2020, as this hasn't happened yet) and is this difference statistically significant from other climate periods?
- Can we already say something about the last 10 years, i.e. the period of 2010 - 2019 (which have seen significant warming and change in weather temperatures in the NL, see for example: https://www.knmi.nl/over-het-knmi/nieuws/zonnige-maand-past-in-trend-voorjaar)
- Following the link posted above, can we say anything significant about weather pattern changes (mainly sunshine hours and precipitation) in early spring? (For non-Dutch speakers, it basically claims the spring has become more sunnier and drier over the last years, showcasing trends).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [2]:
# Get data, available from here: https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/daggegevens/etmgeg_260.zip
daily_data = pd.read_csv("etmgeg_260.txt",sep=",",header=0,skiprows=list(range(47)),skipinitialspace=True)

In [3]:
daily_data

,# STN,YYYYMMDD,DDVEC,FHVEC,FG,FHX,FHXH,FHN,FHNH,FXX,...,VVNH,VVX,VVXH,NG,UG,UX,UXH,UN,UNH,EV24
0,260,19010101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,66.0,NaN,NaN,NaN,NaN,NaN
1,260,19010102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,86.0,NaN,NaN,NaN,NaN,NaN
2,260,19010103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,89.0,NaN,NaN,NaN,NaN,NaN
3,260,19010104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,79.0,NaN,NaN,NaN,NaN,NaN
4,260,19010105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,65.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43584,260,20200430,199.0,44.0,48.0,70.0,8.0,30.0,4.0,140.0,...,20.0,83.0,18.0,7.0,70.0,90.0,6.0,40.0,15.0,24.0
43585,260,20200501,231.0,37.0,43.0,70.0,11.0,20.0,20.0,140.0,...,9.0,81.0,15.0,7.0,77.0,88.0,9.0,57.0,16.0,18.0
43586,260,20200502,272.0,31.0,34.0,50.0,10.0,20.0,6.0,110.0,...,24.0,81.0,13.0,6.0,77.0,97.0,24.0,62.0,13.0,21.0
43587,260,20200503,281.0,3.0,17.0,30.0,14.0,10.0,4.0,70.0,...,4.0,83.0,17.0,7.0,80.0,99.0,2.0,49.0,13.0,21.0


In [4]:
# quick preprocessing; selecting relevant fields only and creating a datetime column
df = daily_data[['YYYYMMDD', 'TG', 'SQ', 'SP', 'DR', 'RH']]

mapper = {
    'YYYYMMDD' : 'DATE',
    'TG' : 'Temperature', #Daily mean temperature, in 0.1 degrees
    'SQ' : 'Sunshine_hours', #Daily hours of sunshine, in 0.1 hours
    'SP' : 'Sunshine_perc', #Daily percentage of sunshine hours compared to total day-length (varies per day)
    'DR' : 'Precipation_duration', #Duration of precipation per day in 0.1 hour
    'RH' : 'Precipation_amount' #Amount of precipatition in 0.1 mm
}

df = df.rename(mapper=mapper, axis=1)
df.index = pd.to_datetime(df['DATE'].astype(str), format='%Y%m%d')
df = df.drop("DATE", axis=1)

In [5]:
df.head()

,Temperature,Sunshine_hours,Sunshine_perc,Precipation_duration,Precipation_amount
DATE,,,,,
1901-01-01,-49,28.0,36.0,NaN,NaN
1901-01-02,-18,0.0,0.0,NaN,NaN
1901-01-03,-26,0.0,0.0,NaN,NaN
1901-01-04,-65,0.0,0.0,NaN,NaN
1901-01-05,-60,36.0,46.0,NaN,NaN


### Completeness of timeseries

In [6]:
print(df.shape)

diff_days = df.index.values[-1] - df.index.values[0]
diff_days = diff_days/np.timedelta64(1,'D')

assert df.shape[0] == diff_days + 1, "Days are missing!"

(43589, 5)


### Missing values

In [7]:
df.isnull().sum()

Temperature                 0
Sunshine_hours             30
Sunshine_perc              30
Precipation_duration    10622
Precipation_amount       1856
dtype: int64

In [8]:
df[df.Precipation_amount.isnull()]

,Temperature,Sunshine_hours,Sunshine_perc,Precipation_duration,Precipation_amount
DATE,,,,,
1901-01-01,-49,28.0,36.0,NaN,NaN
1901-01-02,-18,0.0,0.0,NaN,NaN
1901-01-03,-26,0.0,0.0,NaN,NaN
1901-01-04,-65,0.0,0.0,NaN,NaN
1901-01-05,-60,36.0,46.0,NaN,NaN
...,...,...,...,...,...
1945-04-26,130,NaN,NaN,NaN,NaN
1945-04-27,94,NaN,NaN,NaN,NaN
1945-04-28,63,NaN,NaN,NaN,NaN


In [9]:
df[df.Precipation_duration.isnull()]

,Temperature,Sunshine_hours,Sunshine_perc,Precipation_duration,Precipation_amount
DATE,,,,,
1901-01-01,-49,28.0,36.0,NaN,NaN
1901-01-02,-18,0.0,0.0,NaN,NaN
1901-01-03,-26,0.0,0.0,NaN,NaN
1901-01-04,-65,0.0,0.0,NaN,NaN
1901-01-05,-60,36.0,46.0,NaN,NaN
...,...,...,...,...,...
1945-04-26,130,NaN,NaN,NaN,NaN
1945-04-27,94,NaN,NaN,NaN,NaN
1945-04-28,63,NaN,NaN,NaN,NaN


In [10]:
df[df.Sunshine_hours.isnull()]

,Temperature,Sunshine_hours,Sunshine_perc,Precipation_duration,Precipation_amount
DATE,,,,,
1945-04-01,121,NaN,NaN,NaN,NaN
1945-04-02,113,NaN,NaN,NaN,NaN
1945-04-03,83,NaN,NaN,NaN,NaN
1945-04-04,81,NaN,NaN,NaN,NaN
1945-04-05,78,NaN,NaN,NaN,NaN
1945-04-06,83,NaN,NaN,NaN,NaN
1945-04-07,79,NaN,NaN,NaN,NaN
1945-04-08,69,NaN,NaN,NaN,NaN
1945-04-09,62,NaN,NaN,NaN,NaN


In [11]:
df[df.Sunshine_perc.isnull()]

,Temperature,Sunshine_hours,Sunshine_perc,Precipation_duration,Precipation_amount
DATE,,,,,
1945-04-01,121,NaN,NaN,NaN,NaN
1945-04-02,113,NaN,NaN,NaN,NaN
1945-04-03,83,NaN,NaN,NaN,NaN
1945-04-04,81,NaN,NaN,NaN,NaN
1945-04-05,78,NaN,NaN,NaN,NaN
1945-04-06,83,NaN,NaN,NaN,NaN
1945-04-07,79,NaN,NaN,NaN,NaN
1945-04-08,69,NaN,NaN,NaN,NaN
1945-04-09,62,NaN,NaN,NaN,NaN


In [12]:
# All missing data is from before 19450501, so cutting off the data from before that period
df = df[df.index >= datetime(1945,5,1)]

### Min/max ranges

In [13]:
df.describe()

,Temperature,Sunshine_hours,Sunshine_perc,Precipation_duration,Precipation_amount
count,27398.000000,27398.000000,27398.000000,27398.000000,27398.000000
mean,98.443573,43.551938,33.505803,17.170706,22.271261
std,63.432364,40.821207,29.502605,28.700624,44.941936
min,-149.000000,-1.000000,0.000000,0.000000,-1.000000
25%,54.000000,5.000000,4.000000,0.000000,0.000000
50%,101.000000,34.000000,28.000000,1.000000,1.000000
75%,148.000000,72.000000,58.000000,24.000000,25.000000
max,297.000000,158.000000,96.000000,240.000000,639.000000


- Temperature: mean around 9.8C, Minimum -14.9C, Maximum 29.7C (those are mean daily temperatures!), makes sense
- Sunshine hours: mean around 4.3 hours/day (rainy country....), minimum of -0.1 corresponds to less than 0.05, indication there was some sun but very little in case people need to filter on days any sun (sunshine hours!=0) without messing up the scale. We will set this to 0, maximum is 14.6, seems okay.
- Sunshine perc: mean around 33%, minimum 0%, maximum 96%, seems fine
- Precipation duration: Mean 1.7 hours, Minimum 0 hours (makes sense if no rain), max 24 hours
- Precipation amount: Mean 2.2mm, this would mean yearly of 365*2.2 = 803mm, which is normal for NL, amount of -1 is again less than 0.05mm but still some rain, we will set this to 0 again

In [14]:
df.loc[df.Sunshine_hours==-1, "Sunshine_hours"] = 0
df.loc[df.Precipation_amount==-1, "Precipation_amount"] = 0

# 1. Monthly climate averages

In [15]:
periods = [
    (datetime(1951,1,1), datetime(1980,12,31)),
    (datetime(1961,1,1), datetime(1990,12,31)),
    (datetime(1971,1,1), datetime(2000,12,31)),
    (datetime(1981,1,1), datetime(2010,12,31)),
    (datetime(1991,1,1), datetime(2019,12,31)), #Truncated
]

In [16]:
df['Month'] = df.index.month

In [52]:
averages = pd.DataFrame(index=[x[0] for x in periods], columns = [x for x in range(1,13)])
variances = pd.DataFrame(index=[x[0] for x in periods], columns = [x for x in range(1,13)])

In [53]:
for p in periods:
    averages.loc[p[0],:] = df[(df.index>=p[0]) & (df.index<=p[1])].groupby('Month').agg({'Temperature' : np.mean}).values.flatten().T
    variances.loc[p[0],:] = df[(df.index>=p[0]) & (df.index<=p[1])].groupby('Month').agg({'Temperature' : np.var}).values.flatten().T

In [54]:
averages

,1,2,3,4,5,6,7,8,9,10,11,12
1951-01-01,19.5699,22.8679,48.3054,79.6089,121.343,151.741,165.658,164.402,140.131,102.738,58.1911,32.1441
1961-01-01,21.4269,25.68,50.028,80.1644,122.943,151.728,167.989,166.827,140.401,104.875,59.3322,32.2462
1971-01-01,27.7075,30.1439,57.7269,83.2222,126.895,151.72,173.803,172.103,141.644,102.852,62.2578,39.5968
1981-01-01,30.9312,33.0579,61.6978,91.8756,131.192,156.354,179.284,175.099,144.533,107.224,67.3267,36.6409
1991-01-01,35.2725,37.884,64.733,98.0713,134.418,161.037,182.88,177.611,146.87,109.062,69.7115,41.8432


Wow, that is a very steep increase:

In [55]:
averages.diff()

,1,2,3,4,5,6,7,8,9,10,11,12
1951-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961-01-01,1.85699,2.81212,1.72258,0.555556,1.6,-0.0133333,2.33118,2.42473,0.27,2.13763,1.14111,0.102151
1971-01-01,6.28065,4.46382,7.69892,3.05778,3.95161,-0.00777778,5.81398,5.27634,1.24333,-2.02366,2.92556,7.35054
1981-01-01,3.22366,2.91398,3.97097,8.65333,4.29785,4.63444,5.48065,2.9957,2.88889,4.37204,5.06889,-2.95591
1991-01-01,4.34134,4.82615,3.03519,6.19571,3.22577,4.68234,3.596,2.51175,2.33678,1.83864,2.38483,5.2023


In [56]:
averages.diff().mean(axis=1)

1951-01-01         NaN
1961-01-01    1.411727
1971-01-01    3.835925
1981-01-01    3.795374
1991-01-01    3.681399
dtype: float64

Some observations for the latest period:
- The increase has been positive for every month for the first time
- Warming is most apparent in winter and spring, while late summer and fall show less Warming
- Over all periods, the warming in winter and spring is quite insane, January temperatures increased by 1.6 degrees in total, April saw a warming of 1.8 degrees, where for instance september only saw a warming of 0.6 degrees.

### Statistical difference

Our first attempt will be to compare the last 2 series - if both series from 1981 - 2010 and 1991 - 2019 have the same distribution, then the warming is not significant.

In [57]:
from scipy.stats import ttest_ind, ttest_ind_from_stats

In [61]:
for m in range(1,13):
    mean_old = averages.loc[datetime(1981,1,1),m]
    mean_new = averages.loc[datetime(1991,1,1),m]

    var_old = variances.loc[datetime(1981,1,1),m]
    var_new = variances.loc[datetime(1991,1,1),m]

    obs_old = 30
    obs_new = 29

    print("Month: ",m)
    print(ttest_ind_from_stats(mean1=mean_old,std1=np.sqrt(var_old),nobs1=obs_old,mean2=mean_new,std2=np.sqrt(var_new),nobs2=obs_new,equal_var=False))

Month:  1
Ttest_indResult(statistic=-0.3842530784339425, pvalue=0.7022229887802405)
Month:  2
Ttest_indResult(statistic=-0.45670697218053036, pvalue=0.6496173927372573)
Month:  3
Ttest_indResult(statistic=-0.35205008292691214, pvalue=0.7260994492039499)
Month:  4
Ttest_indResult(statistic=-0.683082617670834, pvalue=0.4973266950460945)
Month:  5
Ttest_indResult(statistic=-0.34837845634650483, pvalue=0.7288431732274043)
Month:  6
Ttest_indResult(statistic=-0.5843304200385286, pvalue=0.5613037812325848)
Month:  7
Ttest_indResult(statistic=-0.46270256765244583, pvalue=0.6453426427980176)
Month:  8
Ttest_indResult(statistic=-0.36036495499446486, pvalue=0.7199065546723964)
Month:  9
Ttest_indResult(statistic=-0.35726006464161053, pvalue=0.7222214689527946)
Month:  10
Ttest_indResult(statistic=-0.21757326630206628, pvalue=0.8285412781455104)
Month:  11
Ttest_indResult(statistic=-0.25508095130402725, pvalue=0.7995787921535232)
Month:  12
Ttest_indResult(statistic=-0.4785289344720486, pvalue=0.

A simple t-test doesn't work here; why not? For the 2 periods, 20 years are fully overlapping meaning that the distribution will be mostly equal and this dilutes the result. What if we would pair the results so that a pair is the same year starting from the measurement period, so for a given month we pair 1981 with 1991, 1982 with 1992 etc.

In [62]:
df['Year'] = df.index.year

In [65]:
from scipy.stats import ttest_rel

for m in range(1,13):
    obs_old = df[(df.index>=datetime(1981,1,1)) & (df.index<=datetime(2009,12,31)) & (df.Month==m)].groupby(['Month','Year']).agg({'Temperature' : np.mean}).values.flatten().T # throw away last year for perfect pairing
    obs_new = df[(df.index>=datetime(1991,1,1)) & (df.index<=datetime(2019,12,31)) & (df.Month==m)].groupby(['Month','Year']).agg({'Temperature' : np.mean}).values.flatten().T

    print("Month: ",m)
    print(ttest_rel(obs_old,obs_new))

Month:  1
Ttest_relResult(statistic=-0.5601911847987997, pvalue=0.5798039024214905)
Month:  2
Ttest_relResult(statistic=-0.7461586162030057, pvalue=0.4617907144936345)
Month:  3
Ttest_relResult(statistic=-0.8131280137708917, pvalue=0.42300527526445064)
Month:  4
Ttest_relResult(statistic=-1.5822084191157304, pvalue=0.12483348425331402)
Month:  5
Ttest_relResult(statistic=-0.6113575615097233, pvalue=0.5458948614266789)
Month:  6
Ttest_relResult(statistic=-2.095315444208517, pvalue=0.04530948517751706)
Month:  7
Ttest_relResult(statistic=-0.8674780084136721, pvalue=0.39305412006907836)
Month:  8
Ttest_relResult(statistic=-0.6078075755277296, pvalue=0.548213685057203)
Month:  9
Ttest_relResult(statistic=-0.6553992723020542, pvalue=0.5175598244401529)
Month:  10
Ttest_relResult(statistic=-0.415322623314054, pvalue=0.6810712671631782)
Month:  11
Ttest_relResult(statistic=-0.4434735486380428, pvalue=0.6608311989026622)
Month:  12
Ttest_relResult(statistic=-0.5868148095946019, pvalue=0.562030

Also this is not very satisfying, we expect for example April to have a very low p-value as the warming has been significant there. Let's look at an extreme and only consider non-overlapping years in the datasets:

In [66]:
from scipy.stats import ttest_rel

for m in range(1,13):
    obs_old = df[(df.index>=datetime(1981,1,1)) & (df.index<=datetime(1990,12,31)) & (df.Month==m)].groupby(['Month','Year']).agg({'Temperature' : np.mean}).values.flatten().T # throw away last year for perfect pairing
    obs_new = df[(df.index>=datetime(2011,1,1)) & (df.index<=datetime(2019,12,31)) & (df.Month==m)].groupby(['Month','Year']).agg({'Temperature' : np.mean}).values.flatten().T

    print("Month: ",m)
    print(ttest_ind(obs_old,obs_new))

Month:  1
Ttest_indResult(statistic=-1.1165184758920743, pvalue=0.2797333539863378)
Month:  2
Ttest_indResult(statistic=-1.1571688933956494, pvalue=0.26320477961741834)
Month:  3
Ttest_indResult(statistic=-0.9457836349184984, pvalue=0.3575090921216949)
Month:  4
Ttest_indResult(statistic=-2.5536321287335104, pvalue=0.02055560296664657)
Month:  5
Ttest_indResult(statistic=-1.3210874640833712, pvalue=0.20397925185496307)
Month:  6
Ttest_indResult(statistic=-2.67096726145429, pvalue=0.01612191944445739)
Month:  7
Ttest_indResult(statistic=-1.7473934434820497, pvalue=0.09860711659475359)
Month:  8
Ttest_indResult(statistic=-1.7641354354371814, pvalue=0.09566937721547089)
Month:  9
Ttest_indResult(statistic=-1.2946022461111208, pvalue=0.21276646587952316)
Month:  10
Ttest_indResult(statistic=-1.1200563583056036, pvalue=0.2782649227419152)
Month:  11
Ttest_indResult(statistic=-1.0986324996138148, pvalue=0.28724498985822544)
Month:  12
Ttest_indResult(statistic=-2.097882911730697, pvalue=0.05

This seems to work much better as  months with most increase show the smallest p-values. However this is not the question we wanted to ask, we now have an answer for the statistical difference of temperatures per month between the 80s and the 10s. It seems t-testing for this purpose simply will not do.

After doing some research it seems that a common method is to split the time-series in non-overlapping parts, compute trends on both and do a statistical difference test on the coefficient to find out if they are statistically significant. I will thus have to reformulate my first question (having answered the first part of expected new normals with the table above already) to a statistically more rigourous explanation:

- is there a significant increase in trend between non-overlapping periods of temperature data for different months?

TBD